In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

Using TensorFlow backend.


In [3]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [5]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['pytorch_geometric_test.pkl',
 'cis-trans-isomerism-feature_test.pkl',
 'pytorch_geometric_train.pkl',
 'laplaction_matrix_train.pkl',
 'feature-engineering-physical-chemical-measurement_test.pkl',
 'laplaction_matrix_test.pkl',
 'cis-trans-isomerism-feature_train.pkl',
 'feature-engineering-physical-chemical-measurement_train.pkl',
 'stanislav-blinov_train.pkl',
 'stanislav-blinov_test.pkl']

In [6]:
mytrial = []

In [ ]:
"""
y=scalar_coupling_constant, type=1
260
{'columns': ['dist_O_0_y', 'tertiary_angle_0', 'max_molecule_atom_1_dist_xyz', 'inv_dist0R', 'dist_N_0_y', 'inv_dist1', 'atom_1_bond_lengths_std', 'tertiary_atom_0', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'atom_index_1_cycle_size_mean', 'tertiary_distance_1', 'dist_to_type_0_mean', 'vander_N.x', 'dist_C_2_x', 'inv_distP', 'vander_H.y', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'adC3', 'bond_atom', 'dist_C_2_y', 'tertiary_distance_4', 'dist_H_1_x', 'cos_f0', 'cos_center0', 'inv_dist1R', 'dist_H_0_x', 'molecule_atom_index_0_dist_mean_div', 'dist_C_1_x', 'dist_C_3_x', 'dist_H_1_y', 'yukawa_H.y', 'inv_distPR', 'eem_0', 'tertiary_angle_1', 'tertiary_angle_2', 'vander_H.x', 'dist_C_0_y', 'tertiary_distance_3', 'dist_1_neg_1', 'dist_C_1_y', 'tertiary_distance_0', 'eem_1', 'dist_N_1_x', 'min_molecule_atom_1_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=4
679
{'columns': ['tertiary_angle_0', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_div', 'cos_c1', 'molecule_atom_index_1_dist_min_diff', 'inv_dist0R', 'tertiary_distance_5', 'atom_1_bond_lengths_std', 'tertiary_atom_0', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'dist_H_2_y', 'atom_index_1_cycle_size_mean', 'dist_O_0_x', 'adC2', 'atom_1_n_bonds', 'sd_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_max', 'adC3', 'bond_atom', 'N1', 'tertiary_distance_4', 'dist_to_type_1_mean', 'dist_C_3_y', 'dist_H_1_x', 'atom_1_bond_lengths_min', 'molecule_atom_index_0_dist_mean_diff', 'molecule_atom_index_0_dist_std_diff', 'cos_f0', 'distN0', 'dist_H_0_x', 'molecule_atom_index_0_dist_mean_div', 'tertiary_angle_3', 'dist_C_1_x', 'dist_C_3_x', 'min_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_max', 'dist_H_1_y', 'inv_distPR', 'eem_0', 'dist_N_1_x', 'tertiary_angle_1', 'tertiary_angle_2', 'cos_c0_c1', 'vander_H.x', 'tertiary_angle_4', 'dist_C_0_y', 'tertiary_distance_3', 'dist_C_1_y', 'eem_1', 'dist_C_0_x', 'cos_c0', 'distance_farthest_0', 'tertiary_atom_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=7
1028
{'columns': ['tertiary_angle_0', 'dist_O_0_y', 'dist_N_0_y', 'cos_c1', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'dist_H_2_y', 'atom_index_1_cycle_size_mean', 'tertiary_distance_1', 'dist_O_0_x', 'adC2', 'dist_C_2_x', 'atom_1_n_bonds', 'dist_H_0_y', 'atom_1_bond_lengths_max', 'dist_N_0_x', 'adC3', 'atom_0_bond_lengths_max', 'dist_C_2_y', 'dist_to_type_1_mean', 'dist_C_3_y', 'tertiary_distance_4', 'dist_H_1_x', 'cos_f0', 'distN0', 'inv_dist1R', 'dist_H_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_3', 'dist_C_1_x', 'dist_C_3_x', 'dist_H_1_y', 'tertiary_angle_1', 'tertiary_angle_2', 'cos_c0_c1', 'dist_C_0_y', 'tertiary_distance_3', 'dist_C_1_y', 'dist_C_0_x', 'cos_c0', 'distance_farthest_0', 'tertiary_atom_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=3
1092
{'columns': ['tertiary_angle_0', 'dist_O_0_y', 'dist_N_0_y', 'cos_c1', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'dist_H_2_y', 'atom_index_1_cycle_size_mean', 'tertiary_distance_1', 'dist_O_0_x', 'adC2', 'dist_C_2_x', 'atom_1_n_bonds', 'dist_H_0_y', 'atom_1_bond_lengths_max', 'dist_N_0_x', 'adC3', 'atom_0_bond_lengths_max', 'dist_C_2_y', 'dist_to_type_1_mean', 'dist_C_3_y', 'tertiary_distance_4', 'dist_H_1_x', 'cos_f0', 'distN0', 'inv_dist1R', 'dist_H_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_3', 'dist_C_1_x', 'dist_C_3_x', 'dist_H_1_y', 'tertiary_angle_1', 'tertiary_angle_2', 'cos_c0_c1', 'dist_C_0_y', 'tertiary_distance_3', 'dist_C_1_y', 'dist_C_0_x', 'cos_c0', 'distance_farthest_0', 'tertiary_atom_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=5
179
{'columns': ['dist_O_1_x', 'cos_f0', 'atom_1_bond_lengths_mean', 'dist_O_0_x', 'tertiary_atom_0', 'is_cis', 'molecule_atom_index_0_dist_max_diff', 'tertiary_atom_1', 'dist_N_0_y', 'max_molecule_atom_0_dist_xyz', 'cos_c1', 'dist_C_2_y', 'tertiary_angle_2', 'cos_c0_c1', 'molecule_atom_index_1_dist_min_diff', 'dist_H_2_y', 'dist_to_type_1_mean', 'eem_1', 'atom_1_n_bonds', 'dist_to_type_mean', 'tertiary_angle_1', 'atom_index_1_n_cycle', 'molecule_atom_1_dist_std_diff', 'molecule_atom_index_0_dist_min_diff', 'atom_0_bond_lengths_max', 'dist_N_0_x', 'dist_O_1_y', 'tertiary_distance_1', 'distance_farthest_0', 'dist_C_3_x', 'dist_C_3_y', 'inv_dist1', 'dist_H_1_y', 'distC0', 'molecule_atom_1_dist_min_diff', 'inv_dist0', 'link0', 'dist_H_0_y', 'cos_c0', 'tertiary_angle_0', 'dist_C_1_y', 'dist_xyz', 'tertiary_angle_3', 'tertiary_distance_5', 'vander_O.y', 'adC3', 'dist_C_1_x', 'tertiary_distance_6', 'molecule_atom_index_1_dist_std_diff', 'atom_1_bond_lengths_max', 'adN1', 'mean_molecule_atom_0_dist_xyz', 'dist_H_0_x', 'tertiary_distance_2', 'dist_C_0_y', 'dihedral', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_0_mean', 'dist_C_2_x', 'eem_0', 'tertiary_distance_4', 'mean_molecule_atom_1_dist_xyz', 'dist_C_0_x', 'inv_dist1R', 'cos_center0_center1', 'atom_index_1_cycle_size_mean', 'tertiary_atom_2', 'inv_dist0R', 'molecule_type_0_dist_std_diff', 'is_trans', 'molecule_atom_index_0_dist_max_div', 'dist_O_0_y', 'adC1', 'tertiary_distance_3', 'molecule_atom_index_0_dist_std_diff', 'adC2', 'dist_C_4_y', 'tertiary_atom_3', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 64}, 'fit': {}}, 'metric': 'mean_absolute_error'},
y=scalar_coupling_constant, type=0
227
{'columns': ['cos_c0_c1',  'tertiary_angle_0',  'cos_c1',  'molecule_atom_index_0_dist_min_diff',  'dist_to_type_mean',  'cos_c0',  'dist_H_0_y',  'dist_O_0_y',  'dist_C_2_y',  'tertiary_atom_1',  'tertiary_angle_1',  'tertiary_distance_2',  'dist_O_0_x',  'dist_C_0_y',  'dist_C_0_x',  'dist_C_1_y',  'max_molecule_atom_1_dist_xyz',  'molecule_atom_index_1_dist_max_div',  'adC3',  'tertiary_distance_1',  'dihedral',  'dist_C_2_x',  'tertiary_distance_3',  'molecule_atom_index_0_dist_max_div',  'molecule_atom_index_0_dist_min_div',  'tertiary_angle_3',  'tertiary_angle_2',  'inv_dist0R',  'dist_to_type_1_mean',  'dist_to_type_std',  'dist_N_0_y',  'dist_H_1_y',  'cos_f0',  'dist_H_0_x',  'distance_c1',  'dist_C_3_y',  'eem_1',  'adC2',  'tertiary_atom_0',  'dist_C_1_x',  'max_distance_y',  'tertiary_atom_2',  'max_molecule_atom_0_dist_xyz',  'dist_H_2_y',  'dist_C_3_x',  'dist_to_type_0_mean',  'dist_xyz',  'cos_f1',  'adN1',  'dist_neg_2',  'distH0',  'atom_1_bond_lengths_mean',  'cos_f0_f1',  'linkM0',  'mean_molecule_atom_0_dist_xyz',  'tertiary_angle_4',  'dist_N_0_x',  'mean_molecule_atom_1_dist_xyz',  'vander_C.y',  'linkM1',  'atom_index_farthest_0',  'adH1',  'dist_H_1_x',  'link0',  'linkN',  'dist_O_1_x',  'inv_dist1R',  'molecule_atom_index_0_dist_max_diff',  'adH2',  'is_cis'], 'cv': {'cls': 'KFold',  'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor',  'init': {'learning_rate': 0.2833769330240482,   'feature_fraction': 0.8818248470204605,   'bagging_fraction': 0.8205197060908092,   'min_data_in_leaf': 202,   'lambda_l1': 0.017039063121824582,   'lambda_l2': 0.8318702431636841,   'max_bin': 100,   'num_leaves': 255,   'random_state': 3895,   'n_jobs': 16},  'fit': {}}, 'metric': 'mean_absolute_error'},
y=scalar_coupling_constant, type=6
227
{'columns': ['dist_H_1_x', 'tertiary_distance_3', 'dist_O_0_x', 'dist_C_2_y', 'dihedral', 'molecule_atom_index_0_dist_min_div', 'cos_f0_f1', 'cos_c1', 'dist_C_0_y', 'inv_dist0R', 'linkN', 'adN1', 'molecule_atom_index_0_dist_max_div', 'inv_dist1R', 'mean_molecule_atom_0_dist_xyz', 'linkM1', 'tertiary_atom_1', 'tertiary_atom_0', 'adH2', 'tertiary_angle_3', 'adC3', 'dist_to_type_0_mean', 'dist_xyz', 'dist_N_0_y', 'dist_C_3_y', 'tertiary_angle_2', 'dist_O_0_y', 'tertiary_angle_0', 'is_cis', 'dist_H_0_y', 'dist_C_0_x', 'cos_f0', 'adC2', 'dist_C_3_x', 'vander_C.y', 'dist_N_0_x', 'dist_C_1_x', 'molecule_atom_index_0_dist_min_diff', 'dist_O_1_x', 'atom_1_bond_lengths_mean', 'dist_C_2_x', 'dist_H_2_y', 'dist_C_1_y', 'tertiary_distance_1', 'tertiary_angle_1', 'tertiary_distance_2', 'distH0', 'dist_to_type_std', 'mean_molecule_atom_1_dist_xyz', 'max_molecule_atom_0_dist_xyz', 'adH1', 'dist_to_type_1_mean', 'dist_H_0_x', 'tertiary_angle_4', 'dist_to_type_mean', 'dist_neg_2', 'link0', 'max_distance_y', 'tertiary_atom_2', 'distance_c1', 'linkM0', 'dist_H_1_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_max_diff', 'cos_f1', 'cos_c0', 'molecule_atom_index_1_dist_max_div', 'max_molecule_atom_1_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
y=scalar_coupling_constant, type=2
227
{'columns': ['dist_O_0_y', 'dist_H_0_x', 'dist_N_0_y', 'tertiary_atom_4', 'mean_molecule_atom_0_dist_xyz', 'cos_c1', 'atom_1_bond_lengths_std', 'atom_1_bond_lengths_min', 'molecule_type_dist_max', 'tertiary_distance_2', 'tertiary_angle_0', 'tertiary_atom_0', 'distance_farthest_0', 'eem_1', 'inv_dist1R', 'atom_index_1_n_cycle', 'vander_O.y', 'tertiary_distance_3', 'dist_C_1_x', 'max_molecule_atom_0_dist_xyz', 'dist_O_0_x', 'dist_N_0_x', 'dist_C_1_y', 'dist_H_1_x', 'tertiary_atom_2', 'adC2', 'yukawa_O.y', 'inv_distPR', 'atom_1_bond_lengths_mean', 'inv_dist1', 'cos_c0_c1', 'tertiary_angle_3', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_max', 'atom_1_n_bonds', 'tertiary_angle_2', 'molecule_atom_index_1_dist_min_div', 'distC0', 'max_molecule_atom_1_dist_xyz', 'dist_C_2_y', 'molecule_atom_index_0_dist_max_div', 'dist_to_type_std', 'dist_C_3_y', 'dist_C_0_y', 'inv_dist0R', 'dist_C_2_x', 'tertiary_atom_1', 'tertiary_angle_4', 'tertiary_atom_3', 'molecule_atom_index_0_dist_max_diff', 'inv_dist0', 'tertiary_distance_6', 'cos_f0', 'dist_C_3_x', 'tertiary_angle_1', 'cos_c0', 'atom_index_1_cycle_size_mean', 'dist_C_4_y', 'tertiary_distance_1', 'adN1', 'adC1', 'dist_H_1_y', 'dist_C_0_x', 'molecule_atom_index_1_dist_min_diff', 'adC3', 'dist_O_1_x', 'dist_H_0_y', 'min_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}

"""

In [9]:
# , 'fc', 'sd','pso','dso'
for y in ['scalar_coupling_constant']:
    for t in [2, 5]:
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
#         n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
#         df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        
        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

In [10]:
df_train=pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test2.gzde', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [11]:
columns_list = [
    {'columns': ['cos_c0_c1',  'tertiary_angle_0',  'cos_c1',  'molecule_atom_index_0_dist_min_diff',  'dist_to_type_mean',  'cos_c0',  'dist_H_0_y',  'dist_O_0_y',  'dist_C_2_y',  'tertiary_atom_1',  'tertiary_angle_1',  'tertiary_distance_2',  'dist_O_0_x',  'dist_C_0_y',  'dist_C_0_x',  'dist_C_1_y',  'max_molecule_atom_1_dist_xyz',  'molecule_atom_index_1_dist_max_div',  'adC3',  'tertiary_distance_1',  'dihedral',  'dist_C_2_x',  'tertiary_distance_3',  'molecule_atom_index_0_dist_max_div',  'molecule_atom_index_0_dist_min_div',  'tertiary_angle_3',  'tertiary_angle_2',  'inv_dist0R',  'dist_to_type_1_mean',  'dist_to_type_std',  'dist_N_0_y',  'dist_H_1_y',  'cos_f0',  'dist_H_0_x',  'distance_c1',  'dist_C_3_y',  'eem_1',  'adC2',  'tertiary_atom_0',  'dist_C_1_x',  'max_distance_y',  'tertiary_atom_2',  'max_molecule_atom_0_dist_xyz',  'dist_H_2_y',  'dist_C_3_x',  'dist_to_type_0_mean',  'dist_xyz',  'cos_f1',  'adN1',  'dist_neg_2',  'distH0',  'atom_1_bond_lengths_mean',  'cos_f0_f1',  'linkM0',  'mean_molecule_atom_0_dist_xyz',  'tertiary_angle_4',  'dist_N_0_x',  'mean_molecule_atom_1_dist_xyz',  'vander_C.y',  'linkM1',  'atom_index_farthest_0',  'adH1',  'dist_H_1_x',  'link0',  'linkN',  'dist_O_1_x',  'inv_dist1R',  'molecule_atom_index_0_dist_max_diff',  'adH2',  'is_cis'], 'cv': {'cls': 'KFold',  'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor',  'init': {'learning_rate': 0.2833769330240482,   'feature_fraction': 0.8818248470204605,   'bagging_fraction': 0.8205197060908092,   'min_data_in_leaf': 202,   'lambda_l1': 0.017039063121824582,   'lambda_l2': 0.8318702431636841,   'max_bin': 100,   'num_leaves': 255,   'random_state': 3895,   'n_jobs': 16},  'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['dist_O_0_y', 'tertiary_angle_0', 'max_molecule_atom_1_dist_xyz', 'inv_dist0R', 'dist_N_0_y', 'inv_dist1', 'atom_1_bond_lengths_std', 'tertiary_atom_0', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'atom_index_1_cycle_size_mean', 'tertiary_distance_1', 'dist_to_type_0_mean', 'vander_N.x', 'dist_C_2_x', 'inv_distP', 'vander_H.y', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'adC3', 'bond_atom', 'dist_C_2_y', 'tertiary_distance_4', 'dist_H_1_x', 'cos_f0', 'cos_center0', 'inv_dist1R', 'dist_H_0_x', 'molecule_atom_index_0_dist_mean_div', 'dist_C_1_x', 'dist_C_3_x', 'dist_H_1_y', 'yukawa_H.y', 'inv_distPR', 'eem_0', 'tertiary_angle_1', 'tertiary_angle_2', 'vander_H.x', 'dist_C_0_y', 'tertiary_distance_3', 'dist_1_neg_1', 'dist_C_1_y', 'tertiary_distance_0', 'eem_1', 'dist_N_1_x', 'min_molecule_atom_1_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['dist_O_0_y', 'dist_H_0_x', 'dist_N_0_y', 'tertiary_atom_4', 'mean_molecule_atom_0_dist_xyz', 'cos_c1', 'atom_1_bond_lengths_std', 'atom_1_bond_lengths_min', 'molecule_type_dist_max', 'tertiary_distance_2', 'tertiary_angle_0', 'tertiary_atom_0', 'distance_farthest_0', 'eem_1', 'inv_dist1R', 'atom_index_1_n_cycle', 'vander_O.y', 'tertiary_distance_3', 'dist_C_1_x', 'max_molecule_atom_0_dist_xyz', 'dist_O_0_x', 'dist_N_0_x', 'dist_C_1_y', 'dist_H_1_x', 'tertiary_atom_2', 'adC2', 'yukawa_O.y', 'inv_distPR', 'atom_1_bond_lengths_mean', 'inv_dist1', 'cos_c0_c1', 'tertiary_angle_3', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_max', 'atom_1_n_bonds', 'tertiary_angle_2', 'molecule_atom_index_1_dist_min_div', 'distC0', 'max_molecule_atom_1_dist_xyz', 'dist_C_2_y', 'molecule_atom_index_0_dist_max_div', 'dist_to_type_std', 'dist_C_3_y', 'dist_C_0_y', 'inv_dist0R', 'dist_C_2_x', 'tertiary_atom_1', 'tertiary_angle_4', 'tertiary_atom_3', 'molecule_atom_index_0_dist_max_diff', 'inv_dist0', 'tertiary_distance_6', 'cos_f0', 'dist_C_3_x', 'tertiary_angle_1', 'cos_c0', 'atom_index_1_cycle_size_mean', 'dist_C_4_y', 'tertiary_distance_1', 'adN1', 'adC1', 'dist_H_1_y', 'dist_C_0_x', 'molecule_atom_index_1_dist_min_diff', 'adC3', 'dist_O_1_x', 'dist_H_0_y', 'min_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['tertiary_angle_0', 'dist_O_0_y', 'dist_N_0_y', 'cos_c1', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'dist_H_2_y', 'atom_index_1_cycle_size_mean', 'tertiary_distance_1', 'dist_O_0_x', 'adC2', 'dist_C_2_x', 'atom_1_n_bonds', 'dist_H_0_y', 'atom_1_bond_lengths_max', 'dist_N_0_x', 'adC3', 'atom_0_bond_lengths_max', 'dist_C_2_y', 'dist_to_type_1_mean', 'dist_C_3_y', 'tertiary_distance_4', 'dist_H_1_x', 'cos_f0', 'distN0', 'inv_dist1R', 'dist_H_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_3', 'dist_C_1_x', 'dist_C_3_x', 'dist_H_1_y', 'tertiary_angle_1', 'tertiary_angle_2', 'cos_c0_c1', 'dist_C_0_y', 'tertiary_distance_3', 'dist_C_1_y', 'dist_C_0_x', 'cos_c0', 'distance_farthest_0', 'tertiary_atom_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['tertiary_angle_0', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_div', 'cos_c1', 'molecule_atom_index_1_dist_min_diff', 'inv_dist0R', 'tertiary_distance_5', 'atom_1_bond_lengths_std', 'tertiary_atom_0', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'dist_H_2_y', 'atom_index_1_cycle_size_mean', 'dist_O_0_x', 'adC2', 'atom_1_n_bonds', 'sd_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_max', 'adC3', 'bond_atom', 'N1', 'tertiary_distance_4', 'dist_to_type_1_mean', 'dist_C_3_y', 'dist_H_1_x', 'atom_1_bond_lengths_min', 'molecule_atom_index_0_dist_mean_diff', 'molecule_atom_index_0_dist_std_diff', 'cos_f0', 'distN0', 'dist_H_0_x', 'molecule_atom_index_0_dist_mean_div', 'tertiary_angle_3', 'dist_C_1_x', 'dist_C_3_x', 'min_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_max', 'dist_H_1_y', 'inv_distPR', 'eem_0', 'dist_N_1_x', 'tertiary_angle_1', 'tertiary_angle_2', 'cos_c0_c1', 'vander_H.x', 'tertiary_angle_4', 'dist_C_0_y', 'tertiary_distance_3', 'dist_C_1_y', 'eem_1', 'dist_C_0_x', 'cos_c0', 'distance_farthest_0', 'tertiary_atom_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['dist_O_1_x', 'cos_f0', 'atom_1_bond_lengths_mean', 'dist_O_0_x', 'tertiary_atom_0', 'is_cis', 'molecule_atom_index_0_dist_max_diff', 'tertiary_atom_1', 'dist_N_0_y', 'max_molecule_atom_0_dist_xyz', 'cos_c1', 'dist_C_2_y', 'tertiary_angle_2', 'cos_c0_c1', 'molecule_atom_index_1_dist_min_diff', 'dist_H_2_y', 'dist_to_type_1_mean', 'eem_1', 'atom_1_n_bonds', 'dist_to_type_mean', 'tertiary_angle_1', 'atom_index_1_n_cycle', 'molecule_atom_1_dist_std_diff', 'molecule_atom_index_0_dist_min_diff', 'atom_0_bond_lengths_max', 'dist_N_0_x', 'dist_O_1_y', 'tertiary_distance_1', 'distance_farthest_0', 'dist_C_3_x', 'dist_C_3_y', 'inv_dist1', 'dist_H_1_y', 'distC0', 'molecule_atom_1_dist_min_diff', 'inv_dist0', 'link0', 'dist_H_0_y', 'cos_c0', 'tertiary_angle_0', 'dist_C_1_y', 'dist_xyz', 'tertiary_angle_3', 'tertiary_distance_5', 'vander_O.y', 'adC3', 'dist_C_1_x', 'tertiary_distance_6', 'molecule_atom_index_1_dist_std_diff', 'atom_1_bond_lengths_max', 'adN1', 'mean_molecule_atom_0_dist_xyz', 'dist_H_0_x', 'tertiary_distance_2', 'dist_C_0_y', 'dihedral', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_0_mean', 'dist_C_2_x', 'eem_0', 'tertiary_distance_4', 'mean_molecule_atom_1_dist_xyz', 'dist_C_0_x', 'inv_dist1R', 'cos_center0_center1', 'atom_index_1_cycle_size_mean', 'tertiary_atom_2', 'inv_dist0R', 'molecule_type_0_dist_std_diff', 'is_trans', 'molecule_atom_index_0_dist_max_div', 'dist_O_0_y', 'adC1', 'tertiary_distance_3', 'molecule_atom_index_0_dist_std_diff', 'adC2', 'dist_C_4_y', 'tertiary_atom_3', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 64}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['dist_H_1_x', 'tertiary_distance_3', 'dist_O_0_x', 'dist_C_2_y', 'dihedral', 'molecule_atom_index_0_dist_min_div', 'cos_f0_f1', 'cos_c1', 'dist_C_0_y', 'inv_dist0R', 'linkN', 'adN1', 'molecule_atom_index_0_dist_max_div', 'inv_dist1R', 'mean_molecule_atom_0_dist_xyz', 'linkM1', 'tertiary_atom_1', 'tertiary_atom_0', 'adH2', 'tertiary_angle_3', 'adC3', 'dist_to_type_0_mean', 'dist_xyz', 'dist_N_0_y', 'dist_C_3_y', 'tertiary_angle_2', 'dist_O_0_y', 'tertiary_angle_0', 'is_cis', 'dist_H_0_y', 'dist_C_0_x', 'cos_f0', 'adC2', 'dist_C_3_x', 'vander_C.y', 'dist_N_0_x', 'dist_C_1_x', 'molecule_atom_index_0_dist_min_diff', 'dist_O_1_x', 'atom_1_bond_lengths_mean', 'dist_C_2_x', 'dist_H_2_y', 'dist_C_1_y', 'tertiary_distance_1', 'tertiary_angle_1', 'tertiary_distance_2', 'distH0', 'dist_to_type_std', 'mean_molecule_atom_1_dist_xyz', 'max_molecule_atom_0_dist_xyz', 'adH1', 'dist_to_type_1_mean', 'dist_H_0_x', 'tertiary_angle_4', 'dist_to_type_mean', 'dist_neg_2', 'link0', 'max_distance_y', 'tertiary_atom_2', 'distance_c1', 'linkM0', 'dist_H_1_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_max_diff', 'cos_f1', 'cos_c0', 'molecule_atom_index_1_dist_max_div', 'max_molecule_atom_1_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['tertiary_angle_0', 'dist_O_0_y', 'dist_N_0_y', 'cos_c1', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'dist_H_2_y', 'atom_index_1_cycle_size_mean', 'tertiary_distance_1', 'dist_O_0_x', 'adC2', 'dist_C_2_x', 'atom_1_n_bonds', 'dist_H_0_y', 'atom_1_bond_lengths_max', 'dist_N_0_x', 'adC3', 'atom_0_bond_lengths_max', 'dist_C_2_y', 'dist_to_type_1_mean', 'dist_C_3_y', 'tertiary_distance_4', 'dist_H_1_x', 'cos_f0', 'distN0', 'inv_dist1R', 'dist_H_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_3', 'dist_C_1_x', 'dist_C_3_x', 'dist_H_1_y', 'tertiary_angle_1', 'tertiary_angle_2', 'cos_c0_c1', 'dist_C_0_y', 'tertiary_distance_3', 'dist_C_1_y', 'dist_C_0_x', 'cos_c0', 'distance_farthest_0', 'tertiary_atom_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
]

In [24]:
columns_list = [
{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'distC1', 'dist_xyz', 'dist_C_2_y', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'distC0', 'vander_H.x', 'dist_H_2_x', 'adC2', 'yukawa_H.y', 'molecule_atom_index_1_dist_min_diff', 'cos_center0', 'dist_C_4_x', 'dist_N_0_x', 'atom_index_farthest_0', 'dist_H_1_y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 10}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['dist_O_0_y', 'tertiary_angle_0', 'max_molecule_atom_1_dist_xyz', 'inv_dist0R', 'dist_N_0_y', 'inv_dist1', 'atom_1_bond_lengths_std', 'tertiary_atom_0', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'atom_index_1_cycle_size_mean', 'tertiary_distance_1', 'dist_to_type_0_mean', 'vander_N.x', 'dist_C_2_x', 'inv_distP', 'vander_H.y', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'adC3', 'bond_atom', 'dist_C_2_y', 'tertiary_distance_4', 'dist_H_1_x', 'cos_f0', 'cos_center0', 'inv_dist1R', 'dist_H_0_x', 'molecule_atom_index_0_dist_mean_div', 'dist_C_1_x', 'dist_C_3_x', 'dist_H_1_y', 'yukawa_H.y', 'inv_distPR', 'eem_0', 'tertiary_angle_1', 'tertiary_angle_2', 'vander_H.x', 'dist_C_0_y', 'tertiary_distance_3', 'dist_1_neg_1', 'dist_C_1_y', 'tertiary_distance_0', 'eem_1', 'dist_N_1_x', 'min_molecule_atom_1_dist_xyz'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 10}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['dist_O_0_y', 'dist_H_0_x', 'dist_N_0_y', 'tertiary_atom_4', 'mean_molecule_atom_0_dist_xyz', 'cos_c1', 'atom_1_bond_lengths_std', 'atom_1_bond_lengths_min', 'molecule_type_dist_max', 'tertiary_distance_2', 'tertiary_angle_0', 'tertiary_atom_0', 'distance_farthest_0', 'eem_1', 'inv_dist1R', 'atom_index_1_n_cycle', 'vander_O.y', 'tertiary_distance_3', 'dist_C_1_x', 'max_molecule_atom_0_dist_xyz', 'dist_O_0_x', 'dist_N_0_x', 'dist_C_1_y', 'dist_H_1_x', 'tertiary_atom_2', 'adC2', 'yukawa_O.y', 'inv_distPR', 'atom_1_bond_lengths_mean', 'inv_dist1', 'cos_c0_c1', 'tertiary_angle_3', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_max', 'atom_1_n_bonds', 'tertiary_angle_2', 'molecule_atom_index_1_dist_min_div', 'distC0', 'max_molecule_atom_1_dist_xyz', 'dist_C_2_y', 'molecule_atom_index_0_dist_max_div', 'dist_to_type_std', 'dist_C_3_y', 'dist_C_0_y', 'inv_dist0R', 'dist_C_2_x', 'tertiary_atom_1', 'tertiary_angle_4', 'tertiary_atom_3', 'molecule_atom_index_0_dist_max_diff', 'inv_dist0', 'tertiary_distance_6', 'cos_f0', 'dist_C_3_x', 'tertiary_angle_1', 'cos_c0', 'atom_index_1_cycle_size_mean', 'dist_C_4_y', 'tertiary_distance_1', 'adN1', 'adC1', 'dist_H_1_y', 'dist_C_0_x', 'molecule_atom_index_1_dist_min_diff', 'adC3', 'dist_O_1_x', 'dist_H_0_y', 'min_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 10}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 10}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['tertiary_angle_0', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_div', 'cos_c1', 'molecule_atom_index_1_dist_min_diff', 'inv_dist0R', 'tertiary_distance_5', 'atom_1_bond_lengths_std', 'tertiary_atom_0', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'dist_H_2_y', 'atom_index_1_cycle_size_mean', 'dist_O_0_x', 'adC2', 'atom_1_n_bonds', 'sd_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_max', 'adC3', 'bond_atom', 'N1', 'tertiary_distance_4', 'dist_to_type_1_mean', 'dist_C_3_y', 'dist_H_1_x', 'atom_1_bond_lengths_min', 'molecule_atom_index_0_dist_mean_diff', 'molecule_atom_index_0_dist_std_diff', 'cos_f0', 'distN0', 'dist_H_0_x', 'molecule_atom_index_0_dist_mean_div', 'tertiary_angle_3', 'dist_C_1_x', 'dist_C_3_x', 'min_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_max', 'dist_H_1_y', 'inv_distPR', 'eem_0', 'dist_N_1_x', 'tertiary_angle_1', 'tertiary_angle_2', 'cos_c0_c1', 'vander_H.x', 'tertiary_angle_4', 'dist_C_0_y', 'tertiary_distance_3', 'dist_C_1_y', 'eem_1', 'dist_C_0_x', 'cos_c0', 'distance_farthest_0', 'tertiary_atom_1'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 10}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                    {'columns': ['dist_O_1_x', 'cos_f0', 'atom_1_bond_lengths_mean', 'dist_O_0_x', 'tertiary_atom_0', 'is_cis', 'molecule_atom_index_0_dist_max_diff', 'tertiary_atom_1', 'dist_N_0_y', 'max_molecule_atom_0_dist_xyz', 'cos_c1', 'dist_C_2_y', 'tertiary_angle_2', 'cos_c0_c1', 'molecule_atom_index_1_dist_min_diff', 'dist_H_2_y', 'dist_to_type_1_mean', 'eem_1', 'atom_1_n_bonds', 'dist_to_type_mean', 'tertiary_angle_1', 'atom_index_1_n_cycle', 'molecule_atom_1_dist_std_diff', 'molecule_atom_index_0_dist_min_diff', 'atom_0_bond_lengths_max', 'dist_N_0_x', 'dist_O_1_y', 'tertiary_distance_1', 'distance_farthest_0', 'dist_C_3_x', 'dist_C_3_y', 'inv_dist1', 'dist_H_1_y', 'distC0', 'molecule_atom_1_dist_min_diff', 'inv_dist0', 'link0', 'dist_H_0_y', 'cos_c0', 'tertiary_angle_0', 'dist_C_1_y', 'dist_xyz', 'tertiary_angle_3', 'tertiary_distance_5', 'vander_O.y', 'adC3', 'dist_C_1_x', 'tertiary_distance_6', 'molecule_atom_index_1_dist_std_diff', 'atom_1_bond_lengths_max', 'adN1', 'mean_molecule_atom_0_dist_xyz', 'dist_H_0_x', 'tertiary_distance_2', 'dist_C_0_y', 'dihedral', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_0_mean', 'dist_C_2_x', 'eem_0', 'tertiary_distance_4', 'mean_molecule_atom_1_dist_xyz', 'dist_C_0_x', 'inv_dist1R', 'cos_center0_center1', 'atom_index_1_cycle_size_mean', 'tertiary_atom_2', 'inv_dist0R', 'molecule_type_0_dist_std_diff', 'is_trans', 'molecule_atom_index_0_dist_max_div', 'dist_O_0_y', 'adC1', 'tertiary_distance_3', 'molecule_atom_index_0_dist_std_diff', 'adC2', 'dist_C_4_y', 'tertiary_atom_3', 'tertiary_angle_4'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 10}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 64}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['dist_H_1_x', 'tertiary_distance_3', 'dist_O_0_x', 'dist_C_2_y', 'dihedral', 'molecule_atom_index_0_dist_min_div', 'cos_f0_f1', 'cos_c1', 'dist_C_0_y', 'inv_dist0R', 'linkN', 'adN1', 'molecule_atom_index_0_dist_max_div', 'inv_dist1R', 'mean_molecule_atom_0_dist_xyz', 'linkM1', 'tertiary_atom_1', 'tertiary_atom_0', 'adH2', 'tertiary_angle_3', 'adC3', 'dist_to_type_0_mean', 'dist_xyz', 'dist_N_0_y', 'dist_C_3_y', 'tertiary_angle_2', 'dist_O_0_y', 'tertiary_angle_0', 'is_cis', 'dist_H_0_y', 'dist_C_0_x', 'cos_f0', 'adC2', 'dist_C_3_x', 'vander_C.y', 'dist_N_0_x', 'dist_C_1_x', 'molecule_atom_index_0_dist_min_diff', 'dist_O_1_x', 'atom_1_bond_lengths_mean', 'dist_C_2_x', 'dist_H_2_y', 'dist_C_1_y', 'tertiary_distance_1', 'tertiary_angle_1', 'tertiary_distance_2', 'distH0', 'dist_to_type_std', 'mean_molecule_atom_1_dist_xyz', 'max_molecule_atom_0_dist_xyz', 'adH1', 'dist_to_type_1_mean', 'dist_H_0_x', 'tertiary_angle_4', 'dist_to_type_mean', 'dist_neg_2', 'link0', 'max_distance_y', 'tertiary_atom_2', 'distance_c1', 'linkM0', 'dist_H_1_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_max_diff', 'cos_f1', 'cos_c0', 'molecule_atom_index_1_dist_max_div', 'max_molecule_atom_1_dist_xyz'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 10}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
    {'columns': ['tertiary_angle_0', 'dist_O_0_y', 'dist_N_0_y', 'cos_c1', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_1_bond_lengths_mean', 'dist_H_2_y', 'atom_index_1_cycle_size_mean', 'tertiary_distance_1', 'dist_O_0_x', 'adC2', 'dist_C_2_x', 'atom_1_n_bonds', 'dist_H_0_y', 'atom_1_bond_lengths_max', 'dist_N_0_x', 'adC3', 'atom_0_bond_lengths_max', 'dist_C_2_y', 'dist_to_type_1_mean', 'dist_C_3_y', 'tertiary_distance_4', 'dist_H_1_x', 'cos_f0', 'distN0', 'inv_dist1R', 'dist_H_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_3', 'dist_C_1_x', 'dist_C_3_x', 'dist_H_1_y', 'tertiary_angle_1', 'tertiary_angle_2', 'cos_c0_c1', 'dist_C_0_y', 'tertiary_distance_3', 'dist_C_1_y', 'dist_C_0_x', 'cos_c0', 'distance_farthest_0', 'tertiary_atom_1'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 10}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
]

In [25]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [26]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-31 13:31:14.387932,modeling for 0,78,0.535713,0.773029,0.237316,modeling for 0
1,2019-07-31 13:33:21.441994,modeling for 3,59,0.107896,0.172881,0.064985,modeling for 3
2,2019-07-31 13:33:34.414431,modeling for 1,46,0.282082,0.417296,0.135213,modeling for 1
3,2019-07-31 13:34:09.022469,modeling for 4,54,0.130426,0.206639,0.076213,modeling for 4
4,2019-07-31 13:38:48.734934,modeling for 2,68,0.242774,0.307655,0.064881,modeling for 2
5,2019-07-31 13:41:37.596538,modeling for 6,68,0.121591,0.177431,0.055839,modeling for 6
6,2019-07-31 14:42:17.058490,modeling for 5,79,0.283279,0.344723,0.061444,modeling for 5
7,2019-07-31 14:43:06.722567,modeling for 7,44,0.080297,0.126018,0.045721,modeling for 7


In [27]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.3550450273270724
0 0.7730294752353206
3 0.17288117709975181
1 0.4172959663856048
4 0.20663846649672102
2 0.30765468740079693
6 0.1774308915338882
5 0.3447227413126608
7 0.12601839693762293


In [22]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.360234434347647
0 0.7842100519689365
3 0.17719210547624356
1 0.42645304414337976
4 0.21421638587900052
2 0.31146407046290264
6 0.18166704194747524
5 0.34845425734081015
7 0.13099211880156642


In [23]:
idx=0.36
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)